# My project for Optimization Course

Multivariate linear regression implemented through solving an optimization problem using the gradient descent method and some of its variants.

The dataset used was taken from Kaggle.com and concerns house prices in the New York area. The idea is to perform a linear regression using the price as the dependent variable and the number of bedrooms, number of bathrooms, and the size of the house as independent variables.

In [2]:
print("hello world")
a=5
b=6
print(a+b)

hello world
11
